# 한국어 파인 튜닝된 Neuron 모델을 SageMaker INF2 에 배포하기

이 노트북은 [MLP-KTLim/llama-3-Korean-Bllossom-8B](https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B) 의 한국어 파인 튜닝 모델을 AWS Neuron Model 로 바꾼 [Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B](https://huggingface.co/Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B) 모델을 사용 합니다.<br>
Amazon EC2 Inferentia2 에 Hugging Face Text Generation Inference (HF TGI) 도커 컨테이너를 이용하여, SageMaker Endpoint 에 배포 합니다.

#### 코드 참조
- [Deploy Llama 3 70B on AWS Inferentia2 with Hugging Face Optimum](https://github.com/philschmid/huggingface-inferentia2-samples/blob/main/llama3-70b/deploy-llama-3-70b-inferentia2.ipynb)



## 1. 다음과 같은 패키지를 설치 합니다.



In [1]:
# !pip install "sagemaker>=2.199.0" "gradio<4" transformers --upgrade --quiet

## 2. SageMaker 세션 등 가져오기

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/SageMaker/.xdg/config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon
sagemaker session region: us-east-1


## 3. Hugging Face LLM Inf2 DLC (Deep Learning Container ) 가져오기

새로운 Hugging Face TGI Neuronx DLC를 사용하여 AWS Inferentia2에서 추론을 실행할 수 있습니다. sagemaker SDK의 get_huggingface_llm_image_uri 메서드를 사용하여 원하는 backend, session, region, 그리고 version에 따라 적절한 Hugging Face TGI Neuronx DLC URI를 검색할 수 있습니다. 사용 가능한 모든 버전은 여기에서 확인할 수 있습니다.
- 참조: [Hugging Face TGI Neuronx DLC URI](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true)



In [3]:
# TODO: Comment in when released
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface-neuronx",
  ## version="0.0.22"
)

print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.23-neuronx-py310-ubuntu22.04


## 4. 한국어 모델을 inferentia2 에 배포하기


모델을 Amazon SageMaker에 배포하기 전에 TGI Neuronx 엔드포인트 구성을 정의해야 합니다. 다음과 같은 추가 매개변수를 정의해야 합니다:

- HF_NUM_CORES: 컴파일에 사용된 Neuron 코어의 수.
- HF_BATCH_SIZE: 모델 컴파일에 사용된 배치 크기.
- HF_SEQUENCE_LENGTH: 모델 컴파일에 사용된 시퀀스 길이.
- HF_AUTO_CAST_TYPE: 모델 컴파일에 사용된 자동 캐스트 유형.

또한 기존의 TGI 매개변수도 정의해야 합니다:

- HF_MODEL_ID: Hugging Face 모델 ID.
- HF_TOKEN: 게이트된 모델에 접근하기 위한 Hugging Face API 토큰. (Llama 모델 사용시 필요)
- MAX_BATCH_SIZE: 모델이 처리할 수 있는 최대 배치 크기로, 컴파일에 사용된 배치 크기와 동일합니다.
- MAX_INPUT_LENGTH: 모델이 처리할 수 있는 최대 입력 길이.
- MAX_TOTAL_TOKENS: 모델이 생성할 수 있는 최대 총 토큰 수로, 컴파일에 사용된 시퀀스 길이와 동일합니다.

##### 옵션: Llama3 모델 등의 HF Token 이 필요시에 주석을 해제하고 사용하세요

In [16]:
# import os
# from getpass import getpass
 
# # is_sagemaker_notebook = True
# is_sagemaker_notebook = False # use VS Code

# if is_sagemaker_notebook:
#     HF_TOKEN = getpass("Enter HUGGINGFACE Access Token: ")
# else: # VS Code
#     from dotenv import load_dotenv
#     HF_TOKEN = os.getenv('HF_TOKEN') or getpass("Enter HUGGINGFACE Access Token: ")
#     print("token: ", HF_TOKEN)


### 모델 배포 파라미터 설정 및 SageMaker HuggingFaceModel 생성

In [6]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.xlarge"
health_check_timeout=1800 # additional time to load the model
volume_size=64 # size in GB of the EBS volume

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B",  
    # "HF_MODEL_ID": "meta-llama/Meta-Llama-3-70B-Instruct",
    # "HF_MODEL_ID": "meta-llama/Meta-Llama-3-8B-Instruct",
    # "HF_MODEL_ID": "meta-llama/Meta-Llama-3.1-8B-Instruct",    
    # "HF_MODEL_ID": "Gonsoo/Llama-3-8b-chat-seqlen-2048-bs-4-neuron-1-8-1",
    "HF_NUM_CORES": "2", # number of neuron cores
    "HF_BATCH_SIZE": "4", # batch size used to compile the model
    "HF_SEQUENCE_LENGTH": "4096", # length used to compile the model
    # "HF_AUTO_CAST_TYPE": "bf16",  # dtype of the model
    "HF_AUTO_CAST_TYPE": "fp16",  # dtype of the model
    "MAX_BATCH_SIZE": "4", # max batch size for the model
    "MAX_INPUT_LENGTH": "4000", # max length of input text
    "MAX_TOTAL_TOKENS": "4096", # max length of generated text
    "MESSAGES_API_ENABLED": "true", # Enable the messages API
    "HF_TOKEN":  "None"# HF_TOKEN
}


# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

### 모델을 SageMaker Endpoint 에 배포

HuggingFaceModel을 생성한 후에는 deploy 메서드를 사용하여 Amazon SageMaker에 배포할 수 있습니다. 우리는 ml.inf2.xlarge 인스턴스 유형을 사용하여 모델을 배포할 것입니다. TGI는 자동으로 모든 Inferentia 디바이스에 걸쳐 모델을 분산하고 샤딩할 것입니다.
- 다음은 약 17분이 소요 됩니다.

In [7]:
# deactivate warning since model is compiled
llm_model._is_compiled_model = True

llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  volume_size=volume_size
)

-------------------------------!

## 5. 추론

엔드포인트가 배포된 후에는 이를 통해 추론을 실행할 수 있습니다. 우리는 predictor의 predict 메서드를 사용하여 엔드포인트에서 추론을 실행할 것입니다. 생성에 영향을 미치는 다양한 매개변수로 추론을 수행할 수 있습니다. 매개변수는 페이로드의 parameters 속성에서 정의할 수 있습니다. 지원되는 매개변수는 여기에서 확인할 수 있습니다.
메시지 API를 사용하면 대화 방식으로 모델과 상호작용할 수 있습니다. 메시지의 역할과 내용을 정의할 수 있습니다. 역할은 system, assistant 또는 user일 수 있습니다. system 역할은 모델에 컨텍스트를 제공하는 데 사용되고, user 역할은 질문을 하거나 모델에 입력을 제공하는 데 사용됩니다.

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What is deep learning?" }
  ]
}
```

In [14]:
# Prompt to generate
messages=[
    { "role": "system", "content": "당신은 인공지능 전문가 입니다." },
    { "role": "user", "content": "딥러닝이 무엇인지 말해 주세요?" }
  ]

# Generation arguments
parameters = {
    "model": "meta-llama/Meta-Llama-3-70B-Instruct", # placholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 2048,
    "stop": ["<|eot_id|>"],
}

이제 테스트 해보시죠

In [15]:
chat = llm.predict({"messages" :messages, **parameters,"steam":True})

print(chat["choices"][0]["message"]["content"].strip())

딥러닝(Deep Learning)은 인공지능(AI)의 한 분야로, 인공신경망(Artificial Neural Network)을 사용하여 데이터를 학습하고 예측하거나 결정을 내리는 기술입니다. 딥러닝은 인공신경망의 다층 구조를 통해 복잡한 패턴을 학습할 수 있게 되어, 인간의 뇌처럼 데이터를 처리하고 학습하는 방식을 모방하고 있습니다.

딥러닝은 다음과 같은 특징을 가지고 있습니다:

1. **다층 구조**: 딥러닝 모델은 여러 층의 인공신경망을 사용하여 데이터를 처리합니다. 각 층은 이전 층에서 추출된 특징을 더 복잡한 특징으로 변환합니다.
2. **자동 특징 추출**: 데이터에서 중요한 특징을 자동으로 추출하여 학습합니다. 이를 통해 인간이 직접 특징을 설계하지 않아도 됩니다.
3. **대규모 데이터 필요**: 딥러닝 모델은 대규모의 데이터를 필요로 합니다. 많은 데이터를 학습함으로써 모델이 더 정확한 예측을 할 수 있게 됩니다.
4. **신경망의 가중치 조정**: 딥러닝 모델은 데이터를 학습하면서 신경망의 가중치를 조정하여 최적의 가중치를 찾습니다. 이 과정을 통해 모델이 더 정확하게 데이터를 이해하게 됩니다.
5. **신경망의 훈련**: 딥러닝 모델은 훈련 과정을 통해 점진적으로 학습됩니다. 훈련 데이터를 사용하여 모델이 예측을 개선하고, 오차를 줄입니다.

딥러닝은 이미지 인식, 음성 인식, 자연어 처리, 게임, 자율 주행 등 다양한 분야에서 널리 사용되고 있습니다. 예를 들어, 이미지 인식 모델은 사람의 얼굴을 인식하거나, 음성 인식 모델은 음성 명령을 이해하고, 자연어 처리 모델은 텍스트를 분석하고 번역할 수 있습니다. 딥러닝은 이러한 복잡한 작업을 자동화하고, 인간의 능력을 넘어서는 성능을 보여주고 있습니다.<|eot_id|>


## 6. 리소스 정리

SageMaker HuggingFace Model 및 엔드포인트를 삭제 합니다.


In [17]:
llm.delete_model()
llm.delete_endpoint()